# _CommonCrawl日本語データの分割抽出

 このNotebookは, CommonCrawl日本語データ分割抽出を行うものです.

GENIACプロジェクトのTeam Hatakeyama(仮)ではCommonCrawlのデータを

質良く加工して日本語LLMの学習に役立つコーパス作成を進めています.

そのコーパス作成で問題となるポイントとして,

CommonCrawlのデータが多すぎることがあります(なんと100TB程度!).


 多くの言語が集まるデータから日本語のデータのみを抜く工程

でもデータ数の多さからチームメンバーだけで行うことは困難です.

そのため, チーム内外でCommonCrawlからの日本語データを分割で抽出し,

最終的に統合するということを目指しています.


## 手順 (Google Colab)..

今回CommonCrawlにおける90000個のアーカイブデータ(warc)を分割で処理します.

90000個を10個ずつのバッチに分けて処理, その結果をGoogleDriveに格納することで,

90000個すべての処理を目指します.

そして, その上で行っていただきたいのは,

1)このセル以降のセルを上から順番に実行していく.

2)最後のセルにあるbatch_numberを変更して, セルを実行
https://colab.research.google.com/drive/1Gq8HQ0iyASH5iOAkosclJEQTwYJvYRmy#scrollTo=UawI0uZgAjz6&line=3&uniqifier=1

3)ファイルメニューに現れる/submit/に保存されている{batch_number}.gzファイルを
ダウンロード

4)バッチの処理が終わったことをgenaic slackなどで共有いただき, アップロードするDriveの場所の指示を受けてください.

60分程度実行すると, 1バッチ終了します.


### 手順 (個別環境 linux)

1)このGoogle Colab Notebookをダウンロードしてください.

2)jupyterが使用できるpython環境で実行してください.

## Environment

pythonのライブラリをインストールします.

※初回だけ実行してください

In [ ]:
!git clone https://github.com/hatakeyama-llm-team/Dataset_CommonCrawl/
import os
path = '/content/Dataset_CommonCrawl'
os.chdir(path)
!ls
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.5 MB/s eta 0:00:00


In [ ]:
import os
path = '/content/Dataset_CommonCrawl/extract_data'
os.chdir(path)
!ls

### Function - File Utils

今回の処理に必要な関数(ファイル処理関係)をインストールします.

※初回だけ実行してください

In [ ]:
import os
from src.file_utils import download_file, decompress_gz
from src.downloader import cc_path_to_urls,download_warc_file,download_warc_file_with_s3
base_url = "https://data.commoncrawl.org/"
os.makedirs("data/gz", exist_ok=True)
os.makedirs("data/warc", exist_ok=True)

### Function - Process Utils

今回の処理に必要な関数(warcファイルの処理)をインストールします.

※初回だけ実行してください

In [ ]:
# Beep when finished
import jupyter_beeper

def beep(frequency=2500):
    b = jupyter_beeper.Beeper()
    b.beep(frequency, secs=1, blocking=True)


## Data Preprocess

### Step1 Download CommonCrawl Paths

CommonCrawlにおけるアーカイブ(warc)が保存されているパスの文字列が

圧縮されたファイル(gz)で保存されている.

このファイルをダウンロードして, 解凍, data/data_list配下に保存する


In [ ]:
from extract_data.warc.download_path_list import download_path_list

"""
download path list from commoncrawl
"""
# Parameter
# 今回処理するwarcのパスリストが圧縮されているURL
# パスリストをダウンロードするフォルダの作成
os.makedirs("data", exist_ok=True)
os.makedirs("data/path_list", exist_ok=True)
download_path_list()

ファイルが正常にダウンロードされました: data/path_list/CC-MAIN-2023-50.gz
data/path_list/CC-MAIN-2023-50.gzが解凍され、data/path_list/CC-MAIN-2023-50に保存されました。


### step2 Check Step1 Result

warcのパスの文字列が保存されているかを確認する

※step1で異常があった際に実行してください

In [1]:
from extract_data.warc.src.downloader import get_cc_path_list

# Process
# 保存されているwarcファイルのパスのリストを取得
path_list = get_cc_path_list(path_dir="data/path_list/*")
# 表示
path_list

data/path_list/CC-MAIN-2023-50


['crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00000.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00001.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00002.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00003.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00004.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00005.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00006.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00007.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231

### Step3 : Download Warc and Extract Japanese Site Data

warc.gzファイルのURI(その一部)を元にwarc.gzファイルをダウンロード/解凍(->warc)する.

warcファイルを読み込み, 内部に保存されている日本語サイトのデータのみを抽出する.

抽出した結果をjson.gz形式で圧縮して保存.

warcファイルは90000個ほど存在するため, この一部のみを処理していく必要あり,

そのため, 処理するファイルのパスをいくつかのbatchに分けている.

このbatchの番号(batch_number)を指定し, そのbatchにおける日本語ページを取得

取得した結果をまとめたzipファイルがsubmit/{batch_number}.zipに保存される.

保存されている内容を指定のGoogleDriveに配置ください

---

#### 行っていただく内容

- is_debugをTrueにして動くかを確認 (初回のみ)

- batch_numberを変更 (取りくむbatchをご指定ください)

- is_debugをFalseにしてデータ抽出/加工スタート

#### 2024/03/04 更新

結果のzipファイルを自動的にダウンロードするように変更いたしました

In [ ]:
from tqdm import tqdm
from extract_data.warc.download_and_parse import download_and_parse
import os
import shutil
import time

from google.colab import files

# 処理結果を自動的にダウンロードするように変更

#@markdown - ここの番号を指定を受けた番号に変更をしてください
#@markdown - batch_numberはコンマ区切りで入力してください
batch_number = 503,504, #@param
is_debug = False

# 保存用ディレクトリの指定
submit_dir = "submit"
# もしdriveがマウントできれば,上のsubmit_dirをコメントアウト, 以下をコードにしてください.
# submit_dir = "/content/drive/MyDrive/CommonCrawl/"
def curation(batch_number, submit_dir="/content/submit", is_debug=False):
    cc_path_list = get_cc_path_list()
    if is_debug:
        n_batch = 3
    else:
        n_batch = 10
    start_idx, end_idx = batch_number * n_batch, (batch_number+1) * n_batch
    target_path_list  = cc_path_list[start_idx:end_idx]
    for cc_path in tqdm(target_path_list):
        download_and_parse(cc_path, f"process/batch{batch_number}")
    shutil.make_archive(f'{submit_dir}/{batch_number}',
                        format='zip', root_dir=f"process/batch{batch_number}")

    shutil.rmtree("process/")

for num in batch_number:
  # batchの番号に従って,データの処理
  curation(num, submit_dir=submit_dir, is_debug=is_debug)

  # ファイルのダウンロード
  try:
    files.download(f"./submit/{num}.zip")
  except:
    print("ERROR: File Download Unsuccessful.")
  time.sleep(10)
  beep()
  shutil.rmtree("/content/data/gz")
  shutil.rmtree("/content/data/warc")
  os.mkdir("/content/data/gz")
  os.mkdir("/content/data/warc")

beep(5000)


data/path_list/CC-MAIN-2023-50


  0%|          | 0/10 [00:00<?, ?it/s]

downloading https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-50/segments/1700679100047.66/warc/CC-MAIN-20231129010302-20231129040302-00530.warc.gz
ファイルが正常にダウンロードされました: data/gz/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00530.warc.gz
decompressing data/gz/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00530.warc.gz
data/gz/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00530.warc.gzが解凍され、data/warc/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00530.warcに保存されました。



0it [00:00, ?it/s]
33it [00:00, 147.20it/s]
72it [00:00, 225.61it/s]
98it [00:00, 189.37it/s]
225it [00:00, 472.64it/s]
311it [00:00, 579.94it/s]
378it [00:00, 538.62it/s]
438it [00:01, 274.87it/s]
527it [00:01, 371.96it/s]
658it [00:01, 544.57it/s]
740it [00:02, 253.29it/s]
804it [00:02, 295.90it/s]
865it [00:02, 334.79it/s]
925it [00:02, 339.01it/s]
1002it [00:02, 406.98it/s]
1125it [00:02, 556.73it/s]
1224it [00:03, 620.54it/s]
1431it [00:03, 948.08it/s]WARNING:bs4.dammit:Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.

1547it [00:03, 570.16it/s]
1637it [00:04, 374.17it/s]
1719it [00:04, 429.59it/s]
1792it [00:04, 463.80it/s]
1862it [00:04, 472.93it/s]
2034it [00:04, 687.11it/s]
2123it [00:04, 672.23it/s]
2214it [00:04, 668.48it/s]
2291it [00:04, 575.66it/s]
2421it [00:05, 695.43it/s]
2511it [00:05, 704.60it/s]
2595it [00:05, 694.92it/s]
2710it [00:05, 802.28it/s]
2796it [00:05, 422.46it/s]
2862it [00:06, 371.04it/s]
2964it [00:06, 468.00it/s]
30

downloading https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-50/segments/1700679100047.66/warc/CC-MAIN-20231129010302-20231129040302-00531.warc.gz
ファイルが正常にダウンロードされました: data/gz/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00531.warc.gz
decompressing data/gz/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00531.warc.gz
data/gz/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00531.warc.gzが解凍され、data/warc/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00531.warcに保存されました。


...


0it [00:00, ?it/s]
108it [00:00, 1066.22it/s]
216it [00:00, 857.72it/s] 
378it [00:00, 1147.54it/s]
498it [00:01, 271.02it/s] 
576it [00:01, 321.93it/s]
651it [00:01, 351.92it/s]
717it [00:03, 105.36it/s]
783it [00:03, 134.41it/s]
834it [00:03, 147.80it/s]
925it [00:04, 212.22it/s]
982it [00:04, 181.17it/s]
1050it [00:04, 227.68it/s]
1099it [00:06, 74.08it/s] 
1215it [00:06, 125.36it/s]WARNING:bs4.dammit:Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.

1271it [00:07, 139.73it/s]
1317it [00:07, 162.73it/s]
1395it [00:07, 199.62it/s]
1437it [00:07, 203.70it/s]
1524it [00:07, 280.43it/s]
1614it [00:07, 372.96it/s]
1680it [00:07, 415.87it/s]
1740it [00:08, 315.56it/s]WARNING:bs4.dammit:Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.

1788it [00:08, 303.20it/s]
1917it [00:08, 405.95it/s]
2088it [00:08, 622.98it/s]
2190it [00:08, 683.92it/s]WARNING:bs4.dammit:Some characters could not be decoded, and were replaced with 

downloading https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-50/segments/1700679100047.66/warc/CC-MAIN-20231129010302-20231129040302-00549.warc.gz
ファイルが正常にダウンロードされました: data/gz/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00549.warc.gz
decompressing data/gz/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00549.warc.gz
data/gz/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00549.warc.gzが解凍され、data/warc/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00549.warcに保存されました。



0it [00:00, ?it/s]
66it [00:00, 524.29it/s]
119it [00:00, 106.28it/s]
145it [00:01, 125.39it/s]
192it [00:01, 162.87it/s]
219it [00:01, 163.87it/s]
310it [00:01, 297.29it/s]
414it [00:01, 306.74it/s]
455it [00:02, 249.06it/s]
582it [00:02, 406.03it/s]
644it [00:02, 222.08it/s]
690it [00:03, 235.50it/s]
735it [00:03, 198.88it/s]WARNING:bs4.dammit:Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.

768it [00:03, 199.62it/s]
828it [00:03, 214.29it/s]
939it [00:03, 344.39it/s]
992it [00:04, 348.61it/s]
1040it [00:04, 353.90it/s]
1227it [00:04, 622.73it/s]WARNING:bs4.dammit:Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.

1302it [00:04, 452.62it/s]
1371it [00:04, 466.03it/s]
1503it [00:04, 629.00it/s]
1583it [00:05, 209.72it/s]
1668it [00:06, 265.95it/s]
1788it [00:06, 333.60it/s]
1853it [00:06, 328.82it/s]
1908it [00:08, 115.22it/s]
1948it [00:08, 117.13it/s]
2091it [00:08, 203.84it/s]
2169it [00:08, 252.82it/s]
2244it [

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>